<h3>Neural network, math experimentation</h3>

In [78]:
# activation functions
# ReLu is very simple, it filters out all negative numbers
# this is a powerful activation function in reality
def activation_ReLu(number):
    if number > 0:
        return number
    else:
        return 0
    
# we also need a derived version of ReLu later
# otherwise the same than original, but instead of original value
# return 1 instead
def activation_ReLu_partial_derivative(number):
    if number > 0:
        return 1
    else:
        return 0

In [ ]:
import random

# initialize weights and biases
# in Keras etc. these are usually randomized in the beginning

# Random number generator:
# Freely adjustable weights and biases arrays
weights = [f'w{i}' for i in range(1, 7)]
biases = [f'bias{i}' for i in range(1, 4)]

# dictionary to hold the values
variables = {}

# random values for weights
for weight in weights:
    variables[weight] = random.uniform(-1, 1) # we'll use ranges from -1 to 1, since this is most fitting for our network

# and for biases
for bias in biases:
    variables[bias] = random.uniform(-1, 1)

# our training data
# x1 = input1, x2 = input2, y = true_value
input1 = 1
input2 = 0
true_value = 2

# our learning rate
LR = 0.01

<b>FORWARD PASS</b>

In [80]:
# NODE 1 OUTPUT
node_1_output = input1 * w1 + input2 * w3 + bias1
node_1_output = activation_ReLu(node_1_output)
node_1_output

0

In [81]:
# NODE 2 OUTPUT
node_2_output = input1 * w2 + input2 * w4 + bias2
node_2_output = activation_ReLu(node_2_output)
node_2_output

0.916833058974408

In [82]:
# NODE 3 OUTPUT
# we can just use Node 1 and 2 outputs, since they
# already contain the the previous weights
node_3_output = node_1_output * w5 + node_2_output * w6 + bias3
node_3_output = activation_ReLu(node_3_output)
node_3_output

1.476905677986447

In [83]:
# compare predicted value with true value
print(f"Predicted: {node_3_output} --> True value: {true_value}")

Predicted: 1.476905677986447 --> True value: 2


In [84]:
# LOSS FUNCTION - we are going to use MSE -> mean squared error
# MSE formula LOSS => (predicted_value - true_value) ^ 2
predicted_value = node_3_output
loss = (predicted_value - true_value) ** 2
loss

0.27362766972281877

<h3>BACKPROPAGATION - update the weights and biases while traversing the network BACKWARDS</h3>

In [85]:
# solving the partial derivative of the loss function with respect to weight 5
deriv_L_w5 = 2 * node_1_output * (node_1_output * w5 + node_2_output * w6 + bias3 - true_value)
deriv_L_w5

-0.0

In [86]:
# this decreases the weight value a little bit
# this is basically our optimizer + learning rate
# this optimizer is known as gradient descent
new_w5 = w5 - LR * deriv_L_w5
new_w5

-0.658763926510477

In [87]:
# solving the partial derivative of the loss function with respect to weight 6
deriv_L_w6 = 2 * node_2_output * (node_1_output * w5 + node_2_output * w6 + bias3 - true_value)
deriv_L_w6

-0.9591803347676598

In [88]:
# calculate new value for weight 6
new_w6 = w6 - LR * deriv_L_w6
new_w6

0.8230405579609998

In [89]:
# solving the partial derivative of the loss function with respect to bias3
# NOTE: the * 1 comes from derivating the bias, which is same as derivating x, 
# which result in 1
deriv_L_b3 = 2 * 1 * (node_1_output * w5 + node_2_output * w6 + bias3 - true_value)
deriv_L_b3

-1.0461886440271062

In [90]:
# update the bias 3 based on previous derivation
new_b3 = bias3 - LR * deriv_L_b3
new_b3

0.7415708544156621

<b>To access the first layer, we need to use chain rule, in order to calculate new values for w1-w4 and bias1/2</b>

In [91]:
# see materials for how we need to split this calculation into two parts
# here we solve the left and right sides separately

# left side is mostly the same as derivating w5 and w6
deriv_L_w1_left = 2 * w5 * (node_1_output * w5 + node_2_output * w6 + bias3 - true_value)

# right side: use ReLu derivation and remember to match correct weights with correct inputs and biases
# based on which weight are you are derivating
# COMPARE THE OTHER ORIGINAL PICTURE IN THE MATERIALS
# in the case of w1 => use w1 and and w3 inside the Relu-derivation, because
# these weights are connected to node 1 (which is connected to w1)
# also use bias1, since it's part of node 1
# finally, multiply all with input1, because it is connected to w1
deriv_L_w1_right = activation_ReLu_partial_derivative(input1 * w1 + input2 * w3 + bias1) * input1
deriv_L_w1 = deriv_L_w1_left * deriv_L_w1_right
new_w1 = w1 - LR * deriv_L_w1
new_w1

-0.2708094937040322

In [92]:
# use the same logic as above, but now from the point of view of w2
# notice how we use w6 and w2/w4 and bias2 in the equation of right side
deriv_L_w2_left = 2 * w6 * (node_1_output * w5 + node_2_output * w6 + bias3 - true_value)
deriv_L_w2_right = activation_ReLu_partial_derivative(input1 * w2 + input2 * w4 + bias2) * input1
deriv_L_w2 = deriv_L_w2_left * deriv_L_w2_right
new_w2 = w2 - LR * deriv_L_w2
new_w2

0.8730296217823986

In [93]:
# use the same formula again
deriv_L_w3_left = 2 * w5 * (node_1_output * w5 + node_2_output * w6 + bias3 - true_value)
deriv_L_w3_right = activation_ReLu_partial_derivative(input1 * w1 + input2 * w3 + bias1) * input2
deriv_L_w3 = deriv_L_w3_left * deriv_L_w3_right
new_w3 = w3 - LR * deriv_L_w3
new_w3

0.375139189073187

In [94]:
# use the same formula again
deriv_L_w4_left = 2 * w6 * (node_1_output * w5 + node_2_output * w6 + bias3 - true_value)
deriv_L_w4_right = activation_ReLu_partial_derivative(input1 * w2 + input2 * w4 + bias2) * input2
deriv_L_w4 = deriv_L_w4_left * deriv_L_w4_right
new_w4 = w4 - LR * deriv_L_w4
new_w4

-0.9925752441255713

In [95]:
# otherwise the same formula, but we can multiply the right side function with just 1
# because it's a derivation of bias-term, which is the same as derivation of x, which results in 1
deriv_L_b1_left = 2 * w5 * (node_1_output * w5 + node_2_output * w6 + bias3 - true_value)
deriv_L_b1_right = activation_ReLu_partial_derivative(input1 * w1 + input2 * w3 + bias1) * 1
deriv_L_b1 = deriv_L_b1_left * deriv_L_b1_right
new_b1 = bias1 - LR * deriv_L_b1
new_b1

-0.8865132116250227

In [96]:
# similar derivation of bias 2
deriv_L_b2_left = 2 * w6 * (node_1_output * w5 + node_2_output * w6 + bias3 - true_value)
deriv_L_b2_right = activation_ReLu_partial_derivative(input1 * w2 + input2 * w4 + bias2) * 1
deriv_L_b2 = deriv_L_b2_left * deriv_L_b2_right
new_b2 = bias2 - LR * deriv_L_b2
new_b2

0.06082385418349838

<b>Everything should be okay now, let's compare the results</b>

In [97]:
print("ORIGINAL WEIGHTS AND BIASES")
print(f"w1: {w1}")
print(f"w2: {w2}")
print(f"w3: {w3}")
print(f"w4: {w4}")
print(f"w5: {w5}")
print(f"w6: {w6}")
print(f"b1: {bias1}")
print(f"b2: {bias2}")
print(f"b3: {bias3}")

print("\n\n######################################")

print("NEW WEIGHTS AND BIASES")
print(f"w1: {new_w1}")
print(f"w2: {new_w2}")
print(f"w3: {new_w3}")
print(f"w4: {new_w4}")
print(f"w5: {new_w5}")
print(f"w6: {new_w6}")
print(f"b1: {new_b1}")
print(f"b2: {new_b2}")
print(f"b3: {new_b3}")



ORIGINAL WEIGHTS AND BIASES
w1: -0.2708094937040322
w2: 0.8645194132866542
w3: 0.375139189073187
w4: -0.9925752441255713
w5: -0.658763926510477
w6: 0.8134487546133232
b1: -0.8865132116250227
b2: 0.052313645687753874
b3: 0.731108967975391


######################################
NEW WEIGHTS AND BIASES
w1: -0.2708094937040322
w2: 0.8730296217823986
w3: 0.375139189073187
w4: -0.9925752441255713
w5: -0.658763926510477
w6: 0.8230405579609998
b1: -0.8865132116250227
b2: 0.06082385418349838
b3: 0.7415708544156621


<b>Trying with new values and increased learning rate

In [98]:
# initialize weights and biases
# in Keras etc. these are usually randomized in the beginning
w1 = new_w1
w2 = new_w2
w3 = new_w3
w4 = new_w4
w5 = new_w5
w6 = new_w6
bias1 = new_b1
bias2 = new_b2
bias3 = new_b3

# our training data
# x1 = input1, x2 = input2, y = true_value
input1 = 1
input2 = 0
true_value = 2

# our learning rate
LR = 0.05

<b>FORWARDS PASS

In [99]:
# NODE 1 OUTPUT
node_1_output = input1 * w1 + input2 * w3 + bias1
node_1_output = activation_ReLu(node_1_output)
node_1_output

0

In [100]:
# NODE 2 OUTPUT
node_2_output = input1 * w2 + input2 * w4 + bias2
node_2_output = activation_ReLu(node_2_output)
node_2_output

0.933853475965897

In [101]:
# NODE 3 OUTPUT
# we can just use Node 1 and 2 outputs, since they
# already contain the the previous weights
node_3_output = node_1_output * w5 + node_2_output * w6 + bias3
node_3_output = activation_ReLu(node_3_output)
node_3_output

1.510170140328453

In [102]:
# compare predicted value with true value
print(f"Predicted: {node_3_output} --> True value: {true_value}")

Predicted: 1.510170140328453 --> True value: 2


In [103]:
# LOSS FUNCTION - we are going to use MSE -> mean squared error
# MSE formula LOSS => (predicted_value - true_value) ^ 2
predicted_value = node_3_output
loss = (predicted_value - true_value) ** 2
loss

0.23993329142584732

<h3>BACKPROPAGATION - update the weights and biases while traversing the network BACKWARDS</h3>

In [104]:
# solving the partial derivative of the loss function with respect to weight 5
deriv_L_w5 = 2 * node_1_output * (node_1_output * w5 + node_2_output * w6 + bias3 - true_value)
deriv_L_w5

-0.0

In [105]:
# this decreases the weight value a little bit
# this is basically our optimizer + learning rate
# this optimizer is known as gradient descent
new_w5 = w5 - LR * deriv_L_w5
new_w5

-0.658763926510477

In [106]:
# solving the partial derivative of the loss function with respect to weight 6
deriv_L_w6 = 2 * node_2_output * (node_1_output * w5 + node_2_output * w6 + bias3 - true_value)
deriv_L_w6

-0.9148586341723233

In [107]:
# calculate new value for weight 6
new_w6 = w6 - LR * deriv_L_w6
new_w6

0.868783489669616

In [108]:
# solving the partial derivative of the loss function with respect to bias3
# NOTE: the * 1 comes from derivating the bias, which is same as derivating x, 
# which result in 1
deriv_L_b3 = 2 * 1 * (node_1_output * w5 + node_2_output * w6 + bias3 - true_value)
deriv_L_b3

-0.9796597193430938

In [109]:
# update the bias 3 based on previous derivation
new_b3 = bias3 - LR * deriv_L_b3
new_b3

0.7905538403828167

<b>To access the first layer, we need to use chain rule, in order to calculate new values for w1-w4 and bias1/2</b>

In [110]:
# see materials for how we need to split this calculation into two parts
# here we solve the left and right sides separately

# left side is mostly the same as derivating w5 and w6
deriv_L_w1_left = 2 * w5 * (node_1_output * w5 + node_2_output * w6 + bias3 - true_value)

# right side: use ReLu derivation and remember to match correct weights with correct inputs and biases
# based on which weight are you are derivating
# COMPARE THE OTHER ORIGINAL PICTURE IN THE MATERIALS
# in the case of w1 => use w1 and and w3 inside the Relu-derivation, because
# these weights are connected to node 1 (which is connected to w1)
# also use bias1, since it's part of node 1
# finally, multiply all with input1, because it is connected to w1
deriv_L_w1_right = activation_ReLu_partial_derivative(input1 * w1 + input2 * w3 + bias1) * input1
deriv_L_w1 = deriv_L_w1_left * deriv_L_w1_right
new_w1 = w1 - LR * deriv_L_w1
new_w1

-0.2708094937040322

In [111]:
# use the same logic as above, but now from the point of view of w2
# notice how we use w6 and w2/w4 and bias2 in the equation of right side
deriv_L_w2_left = 2 * w6 * (node_1_output * w5 + node_2_output * w6 + bias3 - true_value)
deriv_L_w2_right = activation_ReLu_partial_derivative(input1 * w2 + input2 * w4 + bias2) * input1
deriv_L_w2 = deriv_L_w2_left * deriv_L_w2_right
new_w2 = w2 - LR * deriv_L_w2
new_w2

0.9133446058834015

In [112]:
# use the same formula again
deriv_L_w3_left = 2 * w5 * (node_1_output * w5 + node_2_output * w6 + bias3 - true_value)
deriv_L_w3_right = activation_ReLu_partial_derivative(input1 * w1 + input2 * w3 + bias1) * input2
deriv_L_w3 = deriv_L_w3_left * deriv_L_w3_right
new_w3 = w3 - LR * deriv_L_w3
new_w3

0.375139189073187

In [113]:
# use the same formula again
deriv_L_w4_left = 2 * w6 * (node_1_output * w5 + node_2_output * w6 + bias3 - true_value)
deriv_L_w4_right = activation_ReLu_partial_derivative(input1 * w2 + input2 * w4 + bias2) * input2
deriv_L_w4 = deriv_L_w4_left * deriv_L_w4_right
new_w4 = w4 - LR * deriv_L_w4
new_w4

-0.9925752441255713

In [114]:
# otherwise the same formula, but we can multiply the right side function with just 1
# because it's a derivation of bias-term, which is the same as derivation of x, which results in 1
deriv_L_b1_left = 2 * w5 * (node_1_output * w5 + node_2_output * w6 + bias3 - true_value)
deriv_L_b1_right = activation_ReLu_partial_derivative(input1 * w1 + input2 * w3 + bias1) * 1
deriv_L_b1 = deriv_L_b1_left * deriv_L_b1_right
new_b1 = bias1 - LR * deriv_L_b1
new_b1

-0.8865132116250227

In [115]:
# similar derivation of bias 2
deriv_L_b2_left = 2 * w6 * (node_1_output * w5 + node_2_output * w6 + bias3 - true_value)
deriv_L_b2_right = activation_ReLu_partial_derivative(input1 * w2 + input2 * w4 + bias2) * 1
deriv_L_b2 = deriv_L_b2_left * deriv_L_b2_right
new_b2 = bias2 - LR * deriv_L_b2
new_b2

0.10113883828450121

<b>Everything should be okay now, let's compare the results</b>

In [116]:
print("ORIGINAL WEIGHTS AND BIASES")
print(f"w1: {w1}")
print(f"w2: {w2}")
print(f"w3: {w3}")
print(f"w4: {w4}")
print(f"w5: {w5}")
print(f"w6: {w6}")
print(f"b1: {bias1}")
print(f"b2: {bias2}")
print(f"b3: {bias3}")

print("\n\n######################################")

print("NEW WEIGHTS AND BIASES")
print(f"w1: {new_w1}")
print(f"w2: {new_w2}")
print(f"w3: {new_w3}")
print(f"w4: {new_w4}")
print(f"w5: {new_w5}")
print(f"w6: {new_w6}")
print(f"b1: {new_b1}")
print(f"b2: {new_b2}")
print(f"b3: {new_b3}")



ORIGINAL WEIGHTS AND BIASES
w1: -0.2708094937040322
w2: 0.8730296217823986
w3: 0.375139189073187
w4: -0.9925752441255713
w5: -0.658763926510477
w6: 0.8230405579609998
b1: -0.8865132116250227
b2: 0.06082385418349838
b3: 0.7415708544156621


######################################
NEW WEIGHTS AND BIASES
w1: -0.2708094937040322
w2: 0.9133446058834015
w3: 0.375139189073187
w4: -0.9925752441255713
w5: -0.658763926510477
w6: 0.868783489669616
b1: -0.8865132116250227
b2: 0.10113883828450121
b3: 0.7905538403828167


<b>Trying with new values and increased learning rate

In [117]:
# initialize weights and biases
# in Keras etc. these are usually randomized in the beginning
w1 = new_w1
w2 = new_w2
w3 = new_w3
w4 = new_w4
w5 = new_w5
w6 = new_w6
bias1 = new_b1
bias2 = new_b2
bias3 = new_b3

# our training data
# x1 = input1, x2 = input2, y = true_value
input1 = 1
input2 = 0
true_value = 2

# our learning rate
LR = 0.1

<b>FORWARDS PASS

In [118]:
# NODE 1 OUTPUT
node_1_output = input1 * w1 + input2 * w3 + bias1
node_1_output = activation_ReLu(node_1_output)
node_1_output

0

In [119]:
# NODE 2 OUTPUT
node_2_output = input1 * w2 + input2 * w4 + bias2
node_2_output = activation_ReLu(node_2_output)
node_2_output

1.0144834441679027

In [120]:
# NODE 3 OUTPUT
# we can just use Node 1 and 2 outputs, since they
# already contain the the previous weights
node_3_output = node_1_output * w5 + node_2_output * w6 + bias3
node_3_output = activation_ReLu(node_3_output)
node_3_output

1.6719203072190583

In [121]:
# compare predicted value with true value
print(f"Predicted: {node_3_output} --> True value: {true_value}")

Predicted: 1.6719203072190583 --> True value: 2


In [122]:
# LOSS FUNCTION - we are going to use MSE -> mean squared error
# MSE formula LOSS => (predicted_value - true_value) ^ 2
predicted_value = node_3_output
loss = (predicted_value - true_value) ** 2
loss

0.10763628481523711

<h3>BACKPROPAGATION - update the weights and biases while traversing the network BACKWARDS</h3>

In [123]:
# solving the partial derivative of the loss function with respect to weight 5
deriv_L_w5 = 2 * node_1_output * (node_1_output * w5 + node_2_output * w6 + bias3 - true_value)
deriv_L_w5

-0.0

In [124]:
# this decreases the weight value a little bit
# this is basically our optimizer + learning rate
# this optimizer is known as gradient descent
new_w5 = w5 - LR * deriv_L_w5
new_w5

-0.658763926510477

In [125]:
# solving the partial derivative of the loss function with respect to weight 6
deriv_L_w6 = 2 * node_2_output * (node_1_output * w5 + node_2_output * w6 + bias3 - true_value)
deriv_L_w6

-0.6656628333879143

In [126]:
# calculate new value for weight 6
new_w6 = w6 - LR * deriv_L_w6
new_w6

0.9353497730084075

In [127]:
# solving the partial derivative of the loss function with respect to bias3
# NOTE: the * 1 comes from derivating the bias, which is same as derivating x, 
# which result in 1
deriv_L_b3 = 2 * 1 * (node_1_output * w5 + node_2_output * w6 + bias3 - true_value)
deriv_L_b3

-0.6561593855618835

In [128]:
# update the bias 3 based on previous derivation
new_b3 = bias3 - LR * deriv_L_b3
new_b3

0.856169778939005

<b>To access the first layer, we need to use chain rule, in order to calculate new values for w1-w4 and bias1/2</b>

In [129]:
# see materials for how we need to split this calculation into two parts
# here we solve the left and right sides separately

# left side is mostly the same as derivating w5 and w6
deriv_L_w1_left = 2 * w5 * (node_1_output * w5 + node_2_output * w6 + bias3 - true_value)

# right side: use ReLu derivation and remember to match correct weights with correct inputs and biases
# based on which weight are you are derivating
# COMPARE THE OTHER ORIGINAL PICTURE IN THE MATERIALS
# in the case of w1 => use w1 and and w3 inside the Relu-derivation, because
# these weights are connected to node 1 (which is connected to w1)
# also use bias1, since it's part of node 1
# finally, multiply all with input1, because it is connected to w1
deriv_L_w1_right = activation_ReLu_partial_derivative(input1 * w1 + input2 * w3 + bias1) * input1
deriv_L_w1 = deriv_L_w1_left * deriv_L_w1_right
new_w1 = w1 - LR * deriv_L_w1
new_w1

-0.2708094937040322

In [130]:
# use the same logic as above, but now from the point of view of w2
# notice how we use w6 and w2/w4 and bias2 in the equation of right side
deriv_L_w2_left = 2 * w6 * (node_1_output * w5 + node_2_output * w6 + bias3 - true_value)
deriv_L_w2_right = activation_ReLu_partial_derivative(input1 * w2 + input2 * w4 + bias2) * input1
deriv_L_w2 = deriv_L_w2_left * deriv_L_w2_right
new_w2 = w2 - LR * deriv_L_w2
new_w2

0.9703506499601939

In [131]:
# use the same formula again
deriv_L_w3_left = 2 * w5 * (node_1_output * w5 + node_2_output * w6 + bias3 - true_value)
deriv_L_w3_right = activation_ReLu_partial_derivative(input1 * w1 + input2 * w3 + bias1) * input2
deriv_L_w3 = deriv_L_w3_left * deriv_L_w3_right
new_w3 = w3 - LR * deriv_L_w3
new_w3

0.375139189073187

In [132]:
# use the same formula again
deriv_L_w4_left = 2 * w6 * (node_1_output * w5 + node_2_output * w6 + bias3 - true_value)
deriv_L_w4_right = activation_ReLu_partial_derivative(input1 * w2 + input2 * w4 + bias2) * input2
deriv_L_w4 = deriv_L_w4_left * deriv_L_w4_right
new_w4 = w4 - LR * deriv_L_w4
new_w4

-0.9925752441255713

In [133]:
# otherwise the same formula, but we can multiply the right side function with just 1
# because it's a derivation of bias-term, which is the same as derivation of x, which results in 1
deriv_L_b1_left = 2 * w5 * (node_1_output * w5 + node_2_output * w6 + bias3 - true_value)
deriv_L_b1_right = activation_ReLu_partial_derivative(input1 * w1 + input2 * w3 + bias1) * 1
deriv_L_b1 = deriv_L_b1_left * deriv_L_b1_right
new_b1 = bias1 - LR * deriv_L_b1
new_b1

-0.8865132116250227

In [134]:
# similar derivation of bias 2
deriv_L_b2_left = 2 * w6 * (node_1_output * w5 + node_2_output * w6 + bias3 - true_value)
deriv_L_b2_right = activation_ReLu_partial_derivative(input1 * w2 + input2 * w4 + bias2) * 1
deriv_L_b2 = deriv_L_b2_left * deriv_L_b2_right
new_b2 = bias2 - LR * deriv_L_b2
new_b2

0.15814488236129362

<b>Everything should be okay now, let's compare the results</b>

In [135]:
print("ORIGINAL WEIGHTS AND BIASES")
print(f"w1: {w1}")
print(f"w2: {w2}")
print(f"w3: {w3}")
print(f"w4: {w4}")
print(f"w5: {w5}")
print(f"w6: {w6}")
print(f"b1: {bias1}")
print(f"b2: {bias2}")
print(f"b3: {bias3}")

print("\n\n######################################")

print("NEW WEIGHTS AND BIASES")
print(f"w1: {new_w1}")
print(f"w2: {new_w2}")
print(f"w3: {new_w3}")
print(f"w4: {new_w4}")
print(f"w5: {new_w5}")
print(f"w6: {new_w6}")
print(f"b1: {new_b1}")
print(f"b2: {new_b2}")
print(f"b3: {new_b3}")



ORIGINAL WEIGHTS AND BIASES
w1: -0.2708094937040322
w2: 0.9133446058834015
w3: 0.375139189073187
w4: -0.9925752441255713
w5: -0.658763926510477
w6: 0.868783489669616
b1: -0.8865132116250227
b2: 0.10113883828450121
b3: 0.7905538403828167


######################################
NEW WEIGHTS AND BIASES
w1: -0.2708094937040322
w2: 0.9703506499601939
w3: 0.375139189073187
w4: -0.9925752441255713
w5: -0.658763926510477
w6: 0.9353497730084075
b1: -0.8865132116250227
b2: 0.15814488236129362
b3: 0.856169778939005


<b>Trying with new values and increased learning rate

In [136]:
# initialize weights and biases
# in Keras etc. these are usually randomized in the beginning
w1 = new_w1
w2 = new_w2
w3 = new_w3
w4 = new_w4
w5 = new_w5
w6 = new_w6
bias1 = new_b1
bias2 = new_b2
bias3 = new_b3

# our training data
# x1 = input1, x2 = input2, y = true_value
input1 = 1
input2 = 0
true_value = 2

# our learning rate
LR = 0.2

<b>FORWARDS PASS

In [137]:
# NODE 1 OUTPUT
node_1_output = input1 * w1 + input2 * w3 + bias1
node_1_output = activation_ReLu(node_1_output)
node_1_output

0

In [138]:
# NODE 2 OUTPUT
node_2_output = input1 * w2 + input2 * w4 + bias2
node_2_output = activation_ReLu(node_2_output)
node_2_output

1.1284955323214876

In [139]:
# NODE 3 OUTPUT
# we can just use Node 1 and 2 outputs, since they
# already contain the the previous weights
node_3_output = node_1_output * w5 + node_2_output * w6 + bias3
node_3_output = activation_ReLu(node_3_output)
node_3_output

1.9117078189369103

In [140]:
# compare predicted value with true value
print(f"Predicted: {node_3_output} --> True value: {true_value}")

Predicted: 1.9117078189369103 --> True value: 2


In [141]:
# LOSS FUNCTION - we are going to use MSE -> mean squared error
# MSE formula LOSS => (predicted_value - true_value) ^ 2
predicted_value = node_3_output
loss = (predicted_value - true_value) ** 2
loss

0.00779550923687741

<h3>BACKPROPAGATION - update the weights and biases while traversing the network BACKWARDS</h3>

In [142]:
# solving the partial derivative of the loss function with respect to weight 5
deriv_L_w5 = 2 * node_1_output * (node_1_output * w5 + node_2_output * w6 + bias3 - true_value)
deriv_L_w5

-0.0

In [143]:
# this decreases the weight value a little bit
# this is basically our optimizer + learning rate
# this optimizer is known as gradient descent
new_w5 = w5 - LR * deriv_L_w5
new_w5

-0.658763926510477

In [144]:
# solving the partial derivative of the loss function with respect to weight 6
deriv_L_w6 = 2 * node_2_output * (node_1_output * w5 + node_2_output * w6 + bias3 - true_value)
deriv_L_w6

-0.19927466373723307

In [145]:
# calculate new value for weight 6
new_w6 = w6 - LR * deriv_L_w6
new_w6

0.9752047057558542

In [146]:
# solving the partial derivative of the loss function with respect to bias3
# NOTE: the * 1 comes from derivating the bias, which is same as derivating x, 
# which result in 1
deriv_L_b3 = 2 * 1 * (node_1_output * w5 + node_2_output * w6 + bias3 - true_value)
deriv_L_b3

-0.17658436212617934

In [147]:
# update the bias 3 based on previous derivation
new_b3 = bias3 - LR * deriv_L_b3
new_b3

0.8914866513642409

<b>To access the first layer, we need to use chain rule, in order to calculate new values for w1-w4 and bias1/2</b>

In [148]:
# see materials for how we need to split this calculation into two parts
# here we solve the left and right sides separately

# left side is mostly the same as derivating w5 and w6
deriv_L_w1_left = 2 * w5 * (node_1_output * w5 + node_2_output * w6 + bias3 - true_value)

# right side: use ReLu derivation and remember to match correct weights with correct inputs and biases
# based on which weight are you are derivating
# COMPARE THE OTHER ORIGINAL PICTURE IN THE MATERIALS
# in the case of w1 => use w1 and and w3 inside the Relu-derivation, because
# these weights are connected to node 1 (which is connected to w1)
# also use bias1, since it's part of node 1
# finally, multiply all with input1, because it is connected to w1
deriv_L_w1_right = activation_ReLu_partial_derivative(input1 * w1 + input2 * w3 + bias1) * input1
deriv_L_w1 = deriv_L_w1_left * deriv_L_w1_right
new_w1 = w1 - LR * deriv_L_w1
new_w1

-0.2708094937040322

In [149]:
# use the same logic as above, but now from the point of view of w2
# notice how we use w6 and w2/w4 and bias2 in the equation of right side
deriv_L_w2_left = 2 * w6 * (node_1_output * w5 + node_2_output * w6 + bias3 - true_value)
deriv_L_w2_right = activation_ReLu_partial_derivative(input1 * w2 + input2 * w4 + bias2) * input1
deriv_L_w2 = deriv_L_w2_left * deriv_L_w2_right
new_w2 = w2 - LR * deriv_L_w2
new_w2

1.0033842785665052

In [150]:
# use the same formula again
deriv_L_w3_left = 2 * w5 * (node_1_output * w5 + node_2_output * w6 + bias3 - true_value)
deriv_L_w3_right = activation_ReLu_partial_derivative(input1 * w1 + input2 * w3 + bias1) * input2
deriv_L_w3 = deriv_L_w3_left * deriv_L_w3_right
new_w3 = w3 - LR * deriv_L_w3
new_w3

0.375139189073187

In [151]:
# use the same formula again
deriv_L_w4_left = 2 * w6 * (node_1_output * w5 + node_2_output * w6 + bias3 - true_value)
deriv_L_w4_right = activation_ReLu_partial_derivative(input1 * w2 + input2 * w4 + bias2) * input2
deriv_L_w4 = deriv_L_w4_left * deriv_L_w4_right
new_w4 = w4 - LR * deriv_L_w4
new_w4

-0.9925752441255713

In [152]:
# otherwise the same formula, but we can multiply the right side function with just 1
# because it's a derivation of bias-term, which is the same as derivation of x, which results in 1
deriv_L_b1_left = 2 * w5 * (node_1_output * w5 + node_2_output * w6 + bias3 - true_value)
deriv_L_b1_right = activation_ReLu_partial_derivative(input1 * w1 + input2 * w3 + bias1) * 1
deriv_L_b1 = deriv_L_b1_left * deriv_L_b1_right
new_b1 = bias1 - LR * deriv_L_b1
new_b1

-0.8865132116250227

In [153]:
# similar derivation of bias 2
deriv_L_b2_left = 2 * w6 * (node_1_output * w5 + node_2_output * w6 + bias3 - true_value)
deriv_L_b2_right = activation_ReLu_partial_derivative(input1 * w2 + input2 * w4 + bias2) * 1
deriv_L_b2 = deriv_L_b2_left * deriv_L_b2_right
new_b2 = bias2 - LR * deriv_L_b2
new_b2

0.19117851096760488

<b>Everything should be okay now, let's compare the results</b>

In [154]:
print("ORIGINAL WEIGHTS AND BIASES")
print(f"w1: {w1}")
print(f"w2: {w2}")
print(f"w3: {w3}")
print(f"w4: {w4}")
print(f"w5: {w5}")
print(f"w6: {w6}")
print(f"b1: {bias1}")
print(f"b2: {bias2}")
print(f"b3: {bias3}")

print("\n\n######################################")

print("NEW WEIGHTS AND BIASES")
print(f"w1: {new_w1}")
print(f"w2: {new_w2}")
print(f"w3: {new_w3}")
print(f"w4: {new_w4}")
print(f"w5: {new_w5}")
print(f"w6: {new_w6}")
print(f"b1: {new_b1}")
print(f"b2: {new_b2}")
print(f"b3: {new_b3}")



ORIGINAL WEIGHTS AND BIASES
w1: -0.2708094937040322
w2: 0.9703506499601939
w3: 0.375139189073187
w4: -0.9925752441255713
w5: -0.658763926510477
w6: 0.9353497730084075
b1: -0.8865132116250227
b2: 0.15814488236129362
b3: 0.856169778939005


######################################
NEW WEIGHTS AND BIASES
w1: -0.2708094937040322
w2: 1.0033842785665052
w3: 0.375139189073187
w4: -0.9925752441255713
w5: -0.658763926510477
w6: 0.9752047057558542
b1: -0.8865132116250227
b2: 0.19117851096760488
b3: 0.8914866513642409
